In [1]:
%load_ext autoreload
%autoreload 2

import molpy as mp
# import molvis as mv

class Methane(mp.Struct):

    def __init__(self):
        super().__init__()
        C = self.add_atom(name="C", xyz=(0.1, 0, -0.07))
        H1 = self.add_atom(name="H1", xyz=(-0.1, 0, -0.07))
        H2 = self.add_atom(name="H2", xyz=(0., 0.1, 0.07))
        H3 = self.add_atom(name="H3", xyz=(0., -0.1, 0.07))
        self.add_bond(C, H1)
        self.add_bond(C, H2)
        self.add_bond(C, H3)

methane = Methane()
# mp.io.write_pdb("methane.pdb", methane)

In [2]:
class Ethane(mp.Struct):
    def __init__(self):
        super().__init__()
        ch3_1 = mp.builder.CH3()
        ch3_1.translate(-ch3_1["atoms"][0].xyz)
        ch3_2 = mp.builder.CH3()
        ch3_2.translate(-ch3_2["atoms"][0].xyz)
        ch3_2.rotate(180, axis=(0, 1, 0))
        self.add_struct(ch3_1)
        self.add_struct(ch3_2)
        self.add_bond(ch3_1["atoms"][0], ch3_2["atoms"][0])

ethane = Ethane()
# Get topology
topology = ethane.get_topology()

# Calculate angles and dihedrals
angles = ethane.calc_angles(topology)
dihedrals = ethane.calc_dihedrals(topology)

In [3]:
ff = mp.ForceField("opls")
atomstyle = ff.def_atomstyle("full")

# Define atom types
C_type = atomstyle.def_type("C", charge=-0.18, sigma=3.5, epsilon=0.066)
H_type = atomstyle.def_type("H", charge=0.06, sigma=2.5, epsilon=0.03)

# Define bond styles
bondstyle = ff.def_bondstyle("harmonic")
bondstyle.def_type(C_type, C_type, r0=1.54, k=310)
bondstyle.def_type(C_type, H_type, r0=1.09, k=340)

# Define angle styles
anglestyle = ff.def_anglestyle("harmonic")
anglestyle.def_type(H_type, C_type, H_type, theta0=109.5, k=33)
anglestyle.def_type(H_type, C_type, C_type, theta0=110.0, k=35)
anglestyle.def_type(C_type, C_type, C_type, theta0=112.0, k=40)

# Define dihedral styles
dihedralstyle = ff.def_dihedralstyle("opls")
dihedralstyle.def_type(H_type, C_type, C_type, H_type, c0=0.0, c1=1.0, c2=0.0, c3=-0.5)
dihedralstyle.def_type(H_type, C_type, C_type, C_type, c0=0.0, c1=0.5, c2=0.0, c3=-0.25)


<DihedralType: H-C-C-C>

In [4]:
typifier = mp.typifier.Typifier(ff)
typifier.typify_bonds(ethane)

list(ethane.bonds)
ethane.bonds[0]["$type"]["r0"]

1.09

In [5]:
# create ethane mesh
offset = 3.5
structs = []
for i in range(2):
    for j in range(2):
        for k in range(2):
            eth = ethane.copy()
            eth.translate((i*offset, j*offset, k*offset))
            structs.append(eth)

frame = mp.Frame.from_structs(structs)
# mp.io.write_pdb("ethanes.pdb", frame)

In [6]:
frame["bonds"]["type"]

array([1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2])

In [ ]:
class Ethane(mp.Monomer):
    def __init__(self, **props):
        super().__init__(**props)
        ch3_1 = mp.builder.CH3()
        ch3_1.translate(-ch3_1["atoms"][0].xyz)
        ch3_2 = mp.builder.CH3()
        ch3_2.translate(-ch3_2["atoms"][0].xyz)
        ch3_2.rotate(180, axis=(0, 1, 0))
        self.add_struct(ch3_1)
        self.add_struct(ch3_2)
        self.add_bond(ch3_1["atoms"][0], ch3_2["atoms"][0])

ethane = Ethane(
    net_charge=0
)
ethane.def_link_site(
    head=ethane["atoms"][0],
    tail=ethane["atoms"][4],
    delete=ethane["atoms"][1]
)

ambertypifier = mp.typifier.AmberToolsTypifier("gaff", "bcc")
ambertypifier.typify(ethane, workdir=".")

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, abcg2, and sybyl.

-- Check Format for pdb File --


/opt/conda/envs/AmberTools25/bin/wrapped_progs/antechamber: Fatal Error!
Coordinates must be in Columns 31-38, 39-46 and 47-54 in %8.3format.


RuntimeError: Antechamber failed.